# Made by Aleksey Korshuk | https://github.com/AlekseyKorshuk/clip-gan


---


**Star this repository on GitHub: [link](https://github.com/AlekseyKorshuk/clip-gan)**


---



Based on *clip_music_video*

In [1]:
#@title Download and install requirements
%cd /content
!rm /content/clip-gan
!git clone https://github.com/AlekseyKorshuk/clip-gan
%cd /content/clip-gan
!pip install -r requirements.txt
!pip install git+https://github.com/openai/CLIP.git

/content
rm: cannot remove '/content/clip-gan': No such file or directory
Cloning into 'clip-gan'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.
/content/clip-gan
  Cloning git://github.com/openai/CLIP.git to /tmp/pip-req-build-opeujlzl
  Running command git clone -q git://github.com/openai/CLIP.git /tmp/pip-req-build-opeujlzl
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 133kB 7.3MB/s 
     |████████████████████████████████| 7.5MB 8.4MB/s 
     |████████████████████████████████| 184kB 32.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
     |████████████████████████████████| 26.9MB 88kB/s 
     |████████████████████████████████| 276kB 36.9MB/s 
     |████████████████████████████████

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-w_admhfe
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-w_admhfe
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 12.8MB 248kB/s 
  Created wheel for clip: filename=clip-1.0-cp37-none-any.whl size=1368708 sha256=d49577499588e213691dddca3b1ac60d480646a52ac816b43649c59c27ec20fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-ihfwgmd5/wheels/79/51/d7/69f91d37121befe21d9c52332e04f592e17d1cabc7319b3e09
Successfully built clip
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
#@title Restart runtime
import os
os.kill(os.getpid(), 9)



---


# RUN AGAIN FROM THIS PLACE


---



In [1]:
#@title Move to /content/clip-gan
%cd /content/clip-gan

/content/clip-gan


In [2]:
#@title Download video and lyrics
youtube_video_id = "hq3yfQnllfQ" #@param {type:"string"}

from youtube_transcript_api import YouTubeTranscriptApi
import pytube

result = YouTubeTranscriptApi.get_transcript(youtube_video_id)

url = 'https://www.youtube.com/watch?v=' + youtube_video_id

input = "input"
youtube = pytube.YouTube(url)
video = youtube.streams.first()
input_path = video.download(filename=input)

f = open("lyrics.txt", "w")

size = 1000000
if len(result) < size:
  size = len(result)

lines = []
for i in range(0, size):
  cap = result[i]
  if "[" not in cap['text']:
    time = int(cap['start']+cap['duration'])
    min = time//60
    sec = time - min*60
    if len(str(min)) != 2:
      min = "0" +str(min)
    if len(str(sec)) != 2:
      sec = "0" +str(sec)
    line = f"{min}:{sec} {cap['text']}"
    lines.append(line)
    lines.append("\n")

f.writelines(lines[:-1])
f.close()

In [3]:
#@title Get audio from video
!ffmpeg -y -i  input.mp4 -f mp3 -ab 192000 -vn sound.mp3

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [10]:
#@title Generate video
epochs = 10           #@param {type:"integer"}
generator = "biggan"  #@param ["biggan", "dall-e", "stylegan"]
interpol =          30#@param {type:"integer"}


from utils                import train, Pars, create_image, create_outputfolder, init_textfile
from dall_e               import map_pixels, unmap_pixels, load_model
from stylegan             import g_synthesis
from biggan               import BigGAN
from tqdm                 import tqdm
from skimage              import img_as_ubyte
from google.colab.patches import cv2_imshow

import tempfile
import argparse
import torch
import clip
import glob
import os
import math
import cv2
import numpy as np
import warnings

textfile    = "lyrics.txt"
audiofile   = 'sound.mp3'
lyrics      = True
sideX       = 512
sideY       = 512
video_name = "output.avi"

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, interpol, (sideX,sideY))


def main():

    # Automatically creates 'output' folder
    create_outputfolder()

    # Initialize Clip
    perceptor, preprocess   = clip.load('ViT-B/32')
    perceptor               = perceptor.eval()

    # Load the model
    if generator == 'biggan':
        model   = BigGAN.from_pretrained('biggan-deep-512')
        model   = model.cuda().eval()
    elif generator == 'dall-e':
        model   = load_model("https://cdn.openai.com/dall-e/decoder.pkl", "cuda")
    elif generator == 'stylegan':
        model   = g_synthesis.eval().cuda()

    # Read the textfile 
    # descs - list to append the Description and Timestamps
    descs = init_textfile(textfile)

    # list of temporary PTFiles 
    templist = []

    # Loop over the description list
    for d in tqdm(descs):

        timestamp = d[0]
        line = d[1]
        # stamps_descs_list.append((timestamp, line))

        lats = Pars(gen=generator).cuda()

         # Init Generator's latents
        if generator == 'biggan':
            par     = lats.parameters()
            lr      = 0.1#.07
        elif generator == 'stylegan':
            par     = [lats.normu]
            lr      = .01
        elif generator == 'dall-e':
            par     = [lats.normu]
            lr      = .1

        # Init optimizer
        optimizer = torch.optim.Adam(par, lr)

        # tokenize the current description with clip and encode the text
        txt = clip.tokenize(line)
        percep = perceptor.encode_text(txt.cuda()).detach().clone()

        # Training Loop
        for i in range(epochs):
            zs = train(i, model, lats, sideX, sideY, perceptor, percep, optimizer, line, txt, epochs=epochs, gen=generator)

        # save each line's last latent to a torch file temporarily
        latent_temp = tempfile.NamedTemporaryFile()
        torch.save(zs, latent_temp) #f'./output/pt_folder/{line}.pt')
        latent_temp.seek(0)
        #append it to templist so it can be accessed later
        templist.append(latent_temp)
    return templist, descs, model

def sigmoid(x):
    x = x * 2. - 1.
    return math.tanh(1.5*x/(math.sqrt(1.- math.pow(x, 2.)) + 1e-6)) / 2 + .5

def interpolate(templist, descs, model, audiofile):

    video_temp_list = []

    # interpole elements between each image
    current_lyric = ""
    print(descs)
    for idx1, pt in enumerate(descs):

        # get the next index of the descs list, 
        # if it z1_idx is out of range, break the loop
        z1_idx = idx1 + 1
        if z1_idx >= len(descs):
            break

        try:
          current_lyric = descs[idx1+1][1]
        except:
          current_lyric = ""

        print(current_lyric)

        d1 = pt[0]
        d2 = descs[z1_idx][0]
        ttime = d2 - d1

        # if it is the very first index, load the first pt temp file
        # if not assign the previous pt file (z1) to zs variable
        if idx1 == 0:
            zs = torch.load(templist[idx1])
        else:
            zs = z1
        
        # compute for the number of elements to be insert between the 2 elements
        N = round(ttime * interpol)
        # the codes below determine if the output is list (for biggan)
        # if not insert it into a list 
        if not isinstance(zs, list):
            z0 = [zs]
            z1 = [torch.load(templist[z1_idx])]
        else:
            z0 = zs
            z1 = torch.load(templist[z1_idx])
        
        # loop over the range of elements and generate the images
        image_temp_list = []
        for t in range(N):

            azs = []
            for r in zip(z0, z1):
                z_diff = r[1] - r[0] 
                inter_zs = r[0] + sigmoid(t / (N-1)) * z_diff
                azs.append(inter_zs)

            # Generate image
            with torch.no_grad():
                if generator == 'biggan':
                    img = model(azs[0], azs[1], 1).cpu().numpy()
                    img = img[0]
                elif generator == 'dall-e':
                    img = unmap_pixels(torch.sigmoid(model(azs[0])[:, :3]).cpu().float()).numpy()
                    img = img[0]
                elif generator == 'stylegan':
                    img = model(azs[0])
                image_temp = create_image(img, t, current_lyric, generator)
            
            img_last = cv2.imread(image_temp.name + ".png") 
            cv2.putText(img_last, current_lyric, (5, int(sideY*3/4)),
			            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            global video
            video.write(img_last)

if __name__ == '__main__':
    
    warnings.filterwarnings('ignore')
    templist, descs, model = main()
    interpolate(templist, descs, model, audiofile)
    video.release()

100%|██████████| 28/28 [27:03<00:00, 57.99s/it]
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987242221832275]. Convert image to uint8 prior to saving to suppress this warning.


[(-2208988800.0, 'start song'), (-2208988782.0, 'A for Apple, A for Ant'), (-2208988775.0, 'B for Bat, B for Ball'), (-2208988765.0, 'C for Cat and C for Cup'), (-2208988758.0, 'D for Dog, D for Dots'), (-2208988748.0, 'E for Egg and E for Elephant'), (-2208988741.0, 'F for Frog, F for Fish'), (-2208988731.0, 'G for Goat and G for Grass'), (-2208988724.0, 'H for Hand, H for Hat'), (-2208988714.0, 'I for Ink and I for Insect'), (-2208988707.0, 'J for Juice, J for Jar'), (-2208988697.0, 'K for King and K for Kite'), (-2208988690.0, 'L for Lamb, L for Leaf'), (-2208988681.0, 'M for Man and M for Monkey'), (-2208988675.0, 'N for Neck, N for Nose'), (-2208988665.0, 'O for Ocean, O for Oar'), (-2208988659.0, 'P for Plum, P for Parrot'), (-2208988646.0, 'Q for Queen and Q for Quail'), (-2208988640.0, 'R for Rope, R for Rat'), (-2208988630.0, 'S for Sun and S for Sun Flower'), (-2208988623.0, 'T for Tap, T for Tub'), (-2208988614.0, 'U for Uncle, U for Umbrella'), (-2208988607.0, 'V for Violin

Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987242221832275]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987242221832275]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987241625785828]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987242221832275]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987241625785828]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.9987241625785828]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999540448188782, 0.998724

B for Bat, B for Ball


Lossy conversion from float32 to uint8. Range [-0.9999728798866272, 0.9677079319953918]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999728798866272, 0.967707633972168]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999728798866272, 0.9677045345306396]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999728798866272, 0.967689037322998]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9999728798866272, 0.9676467776298523]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.999972939491272, 0.9675597548484802]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.999972939491272, 0.9674148559

...

Lossy conversion from float32 to uint8. Range [-0.9974986910820007, 0.7755120992660522]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974987506866455, 0.7755125164985657]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974984526634216, 0.7755106687545776]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974976778030396, 0.7755044102668762]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.997494637966156, 0.7754843235015869]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974900484085083, 0.7754378318786621]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974730610847473, 0.7753529

Z for Zoo, Z for Zebra


Lossy conversion from float32 to uint8. Range [-0.9996117949485779, 0.9082789421081543]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996117949485779, 0.9082793593406677]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996117949485779, 0.9082796573638916]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996119141578674, 0.9082788825035095]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996120929718018, 0.9082752466201782]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996124505996704, 0.9082745909690857]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9996131062507629, 0.908249

end song


Lossy conversion from float32 to uint8. Range [-0.997458815574646, 0.9997556209564209]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974588751792908, 0.9997556209564209]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974588751792908, 0.9997556209564209]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974581003189087, 0.9997556209564209]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974557757377625, 0.9997555613517761]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974504709243774, 0.9997555017471313]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9974413514137268, 0.9997551

In [11]:
#@title Convert avi to mp4 and concatenate audio file
!ffmpeg -y -i output.avi output.mp4
!ffmpeg -y -i output.mp4 -i sound.mp3 -c copy -map 0:v:0 -map 1:a:0 result.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [12]:
#@title Download result of our generation
from google.colab import files
files.download('result.mp4') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>